In [2]:
import cv2
import numpy as np
import pickle
import matplotlib.pyplot as plt 


def mask_pixels_within_lab_range(frame, lab_colors, tolerance=0.1):
    """
    Given an input frame (BGR) and a set of reference LAB colors, return a binary mask where
    each pixel is selected if its LAB value is within ±(tolerance*100)% of any of the reference colors.
    
    For each reference color, the tolerance is applied per channel. For example, if the reference
    LAB value is [L, a, b], then a pixel's LAB value [L_p, a_p, b_p] is considered close if:
      L_p is in [L*(1-tolerance), L*(1+tolerance)],
      a_p is in [a*(1-tolerance), a*(1+tolerance)], and
      b_p is in [b*(1-tolerance), b*(1+tolerance)].
    
    Parameters:
      frame (numpy.ndarray): Input image in BGR format.
      lab_colors (list or tuple): A list of reference LAB colors. Each color should be an iterable
                                  of three values (L, a, b). The values should be in the same scale
                                  as produced by cv2.cvtColor(..., cv2.COLOR_BGR2LAB) (typically 0–255).
      tolerance (float): Relative tolerance (default 0.1 for 10%).
    
    Returns:
      mask (numpy.ndarray): A binary mask (uint8) where matching pixels are 255 and others 0.
    """
    # Convert the input frame from BGR to LAB color space.
    lab_img = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    
    # Initialize an empty mask (all zeros).
    mask_total = np.zeros(lab_img.shape[:2], dtype=np.uint8)
    
    for color in lab_colors:
        # Convert the reference color to a float32 NumPy array.
        ref = np.array(color, dtype=np.float32)
        
        # Compute lower and upper bounds as ±(tolerance * value) per channel.
        lower_bound = np.clip(ref * (1 - tolerance), 0, 255).astype(np.uint8)
        upper_bound = np.clip(ref * (1 + tolerance), 0, 255).astype(np.uint8)
        print(lower_bound)
        print(upper_bound)
        # Generate a mask for pixels within these bounds.
        mask = cv2.inRange(lab_img, lower_bound, upper_bound)
        
        # Combine the mask with the total mask (bitwise OR).
        mask_total = cv2.bitwise_or(mask_total, mask)
    
    return mask_total


# ========================================================
# Segmentation Model Class
# ========================================================
class SegmentationModel:
    def __init__(self, object_color_lab, background_color_lab, object_area, background_area, object_bbox):
        self.object_color_lab = np.array(object_color_lab, dtype=np.float32)
        self.background_color_lab = np.array(background_color_lab, dtype=np.float32)
        self.object_area = object_area
        self.background_area = background_area
        self.object_bbox = object_bbox

    def update(self, new_object_color_lab, new_background_color_lab, new_object_area, new_background_area, new_object_bbox, alpha=0.1):
        """
        Update the stored cluster centers (and other parameters) using a weighted moving average.
        alpha is the update rate (0 < alpha < 1).
        """
        self.object_color_lab = (1 - alpha) * self.object_color_lab + alpha * np.array(new_object_color_lab, dtype=np.float32)
        self.background_color_lab = (1 - alpha) * self.background_color_lab + alpha * np.array(new_background_color_lab, dtype=np.float32)
        self.object_area = new_object_area  # Optionally, you could update this with a moving average as well.
        self.background_area = new_background_area
        self.object_bbox = new_object_bbox

    def save(self, filename):
        """Save the model to disk using pickle."""
        with open(filename, 'wb') as f:
            pickle.dump(self, f)

    @staticmethod
    def load(filename):
        """Load the model from disk using pickle."""
        with open(filename, 'rb') as f:
            model = pickle.load(f)
        return model

# ========================================================
# Helper Functions to Initialize and Update the Model
# ========================================================
def initialize_segmentation_model(frame, k=2):
    """
    Run the segmentation routine on the frame and initialize a SegmentationModel.
    """
    result = segment_two_colors(frame, k=k)
    model = SegmentationModel(
        object_color_lab=result['object_color_lab'],
        background_color_lab=result['background_color_lab'],
        object_area=result['object_area'],
        background_area=result['background_area'],
        object_bbox=result['object_bbox']
    )
    return model

def update_segmentation_model(frame, model, k=2, alpha=0.1):
    """
    Update an existing segmentation model with a new frame.
    
    Since the ordering of clusters might change between runs of k-means, we
    compare the new centers with the stored ones in the model and assign them accordingly.
    """
    result = segment_two_colors(frame, k=k)
    new_obj_lab = result['object_color_lab']
    new_bg_lab = result['background_color_lab']
    
    # Compare new centers with stored object center.
    # Compute distances in LAB space.
    d_obj_to_new_obj = np.linalg.norm(model.object_color_lab - new_obj_lab)
    d_obj_to_new_bg  = np.linalg.norm(model.object_color_lab - new_bg_lab)
    
    # If the new cluster labeled as "object" is farther from our stored object center than
    # the new cluster labeled as "background", then swap them.
    if d_obj_to_new_bg < d_obj_to_new_obj:
        new_obj_lab, new_bg_lab = new_bg_lab, new_obj_lab
        new_obj_area, new_bg_area = result['background_area'], result['object_area']
        new_obj_bbox = result['object_bbox']  # Assuming object_bbox always corresponds to the object cluster.
    else:
        new_obj_area, new_bg_area = result['object_area'], result['background_area']
        new_obj_bbox = result['object_bbox']
    
    # Update the model using a weighted average update.
    model.update(new_obj_lab, new_bg_lab, new_obj_area, new_bg_area, new_obj_bbox, alpha=alpha)
    return model

# ========================================================
# Example Usage
# ========================================================
if __name__ == '__main__':
    # Assume we have a video file or camera feed.
    cap = cv2.VideoCapture('red_dot_video.mp4')  # Change to 0 for webcam

    # Read the first frame to initialize the model.
    ret, frame = cap.read()
    if not ret:
        print("Error: Unable to capture the first frame.")
        cap.release()
        exit(0)

    # Initialize the segmentation model on the first frame.
    model = initialize_segmentation_model(frame, k=2)
    print("Initial model:")
    print("  Object LAB:", model.object_color_lab)
    print("  Background LAB:", model.background_color_lab)
    print("  Object area:", model.object_area)
    print("  Object bbox:", model.object_bbox)
    
    # Save the initial model for future use.
    model.save('segmentation_model.pkl')
    i = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        #plt.imshow(frame)
        #plt.show()  
        print(i)
        i += 1

        # (Optionally) Load the model from disk if needed:
        # model = SegmentationModel.load('segmentation_model.pkl')

        # Update the model using the current frame.
        #model = update_segmentation_model(frame, model, k=2, alpha=0.1)
        # Print current model parameters.
        #print("Updated model:")
        #print("  Object LAB:", model.object_color_lab)

        gf = mask_pixels_within_lab_range(frame, [(125.487305, 202.85654,  190.72),], tolerance=0.1)
        #plt.imshow(gf)
        #plt.show()  
        # (Optional) Display the updated object mask on the current frame.
        #result = segment_two_colors(frame, k=2)
        #plt.imshow(result['object_mask'])
        #plt.show()

    # Save the updated model at the end.
    model.save('segmentation_model.pkl')
    print("Final model saved.")
    cap.release()


Initial model:
  Object LAB: [125.487305 202.85654  190.72    ]
  Background LAB: [2.0733946e-03 1.2800175e+02 1.2800098e+02]
  Object area: 2600
  Object bbox: (965, 543, 51, 51)
0
[112 182 171]
[138 223 209]
1
[112 182 171]
[138 223 209]
2
[112 182 171]
[138 223 209]
3
[112 182 171]
[138 223 209]
4
[112 182 171]
[138 223 209]
5
[112 182 171]
[138 223 209]
6
[112 182 171]
[138 223 209]
7
[112 182 171]
[138 223 209]
8
[112 182 171]
[138 223 209]
9
[112 182 171]
[138 223 209]
10
[112 182 171]
[138 223 209]
11
[112 182 171]
[138 223 209]
12
[112 182 171]
[138 223 209]
13
[112 182 171]
[138 223 209]
14
[112 182 171]
[138 223 209]
15
[112 182 171]
[138 223 209]
16
[112 182 171]
[138 223 209]
17
[112 182 171]
[138 223 209]
18
[112 182 171]
[138 223 209]
19
[112 182 171]
[138 223 209]
20
[112 182 171]
[138 223 209]
21
[112 182 171]
[138 223 209]
22
[112 182 171]
[138 223 209]
23
[112 182 171]
[138 223 209]
24
[112 182 171]
[138 223 209]
25
[112 182 171]
[138 223 209]
26
[112 182 171]
[138 22